In [1]:
import plunk.sb.sig_compatibility.sig_comp as sc

In [2]:
def foo(a, /,b, *args, c,  d=10,**kwargs):
    pass

In [3]:
d= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)


In [4]:
d

DefinitionSig(PO=1, PK=1, VP=0, KO=2, VK=1)

In [5]:
c = sc.CallSig(pos=1, k=2) # foo(a, b=2, c=3)

In [7]:
d= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)

e= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)

cc = sc.CallSig(pos=2, k=1)

In [8]:
sc.call_is_compatible(d,cc)

False

In [9]:
sc.is_compatible_with(d,e)

True

In [11]:
from i2 import Sig
sig = Sig(foo)

In [19]:
for att in dir(sig):
    if att.startswith('_') or callable(getattr(sig, att)):
        continue
    print(att, getattr(sig,att))

KEYWORD_ONLY KEYWORD_ONLY
POSITIONAL_ONLY POSITIONAL_ONLY
POSITIONAL_OR_KEYWORD POSITIONAL_OR_KEYWORD
VAR_KEYWORD VAR_KEYWORD
VAR_POSITIONAL VAR_POSITIONAL
annotations {}
defaults {'d': 10}
has_var_keyword True
has_var_kinds True
has_var_positional True
index_of_var_keyword 5
index_of_var_positional 2
kinds {'a': <_ParameterKind.POSITIONAL_ONLY: 0>, 'b': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, 'args': <_ParameterKind.VAR_POSITIONAL: 2>, 'c': <_ParameterKind.KEYWORD_ONLY: 3>, 'd': <_ParameterKind.KEYWORD_ONLY: 3>, 'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}
n_required 3
name foo
names ['a', 'b', 'args', 'c', 'd', 'kwargs']
parameters OrderedDict([('a', <Parameter "a">), ('b', <Parameter "b">), ('args', <Parameter "*args">), ('c', <Parameter "c">), ('d', <Parameter "d=10">), ('kwargs', <Parameter "**kwargs">)])
params [<Parameter "a">, <Parameter "b">, <Parameter "*args">, <Parameter "c">, <Parameter "d=10">, <Parameter "**kwargs">]
var_keyword_name kwargs
var_positional_name args


In [27]:
from i2.signatures import PO, PK, VP, KO, VK
from plunk.sb.sig_compatibility.sig_comp import kind_to_symbol_func, transform_key,var_names_by_kind
def invert_dict(d):
    inv_map = {}
    for k, v in d.items():
        inv_map[v] = inv_map.get(v, []) + [k]
    return inv_map

def var_names_by_kind(sig):
    d = invert_dict(sig.kinds)
    func = kind_to_symbol_func
    result = transform_key(d, func)
    return result



In [29]:
var_names_by_kind(sig)

{'PO': ['a'], 'PK': ['b'], 'VP': ['args'], 'KO': ['c', 'd'], 'VK': ['kwargs']}

In [ ]:

def transform_key(d, func):
    return {func(k): v for k, v in d.items()}


kind_to_symbol = {PO: "PO", PK: "PK", VP: "VP", KO: "KO", VK: "VK"}



def kind_to_symbol_func(k):
    return kind_to_symbol[k]
